In [ ]:
import pandas as pd
import numpy as np
import string
import re

In [ ]:
df1= pd.read_csv(r"C:\Users\i\Downloads\fear-ratings-0to1.train.txt", delimiter='\t', header=None)
df1.columns = ['Id', 'tweet', 'emotion', 'score']
df1_dev= pd.read_csv(r"C:\Users\i\Downloads\fear-ratings-0to1.dev.target.txt", delimiter='\t', header=None)
df1_dev.columns = ['Id', 'tweet', 'emotion', 'score']
df1_test= pd.read_csv(r"C:\Users\i\Downloads\fear-ratings-0to1.test.target.txt", delimiter='\t', header=None)
df1_test.columns = ['Id', 'tweet', 'emotion', 'score']

In [ ]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import emoji

In [ ]:
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)
    
    return tweet
df1['tweet'] = df1['tweet'].apply(preprocess_tweet)

In [ ]:
def remove_mentions(tweet):
    return re.sub(r'@\w+', '', tweet)

df1['tweet'] = df1['tweet'].apply(remove_mentions)


In [ ]:
def clean_text(text):
    text = re.sub(r'[^\w\s#@]', '', text)  
    text = re.sub(r'\d+', '', text) 
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df1['tweet'] = df1['tweet'].apply(clean_text)
print(df1)

In [ ]:
def convert_emoji(tweet):
    text = emoji.demojize(tweet)
    return text

In [ ]:
df1['tweet'] = df1['tweet'].apply(convert_emoji)

In [ ]:
def tokenize_tweets(tweet):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(tweet)

df1['tweet'] = df1['tweet'].apply(tokenize_tweets)

In [ ]:
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return filtered_tokens

In [ ]:
df1['tweet'] = df1['tweet'].apply(remove_stopwords)

In [ ]:
df1

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")


In [ ]:
def lemmatize(text_list):
    lemmatized_output = []
    for text in text_list:
        doc = nlp(text)
        lemmatized_output.append(' '.join([token.lemma_ for token in doc]))
    return lemmatized_output

In [ ]:
df1['tweet'] = df1['tweet'].apply(lemmatize)


In [ ]:
def join_sent(text):  
    str1 = " "
    return str1.join(text)
     

In [ ]:
df1['Sentences']=df1['tweet'].apply(join_sent)
df1.head()

In [ ]:
comment_words = ''
for val in df1['tweet']:
    comment_words += " ".join(val)+" "

In [ ]:
len(comment_words)
li = list(comment_words.split(" "))
x=np.array(li)
max_words=len(np.unique(x))

In [ ]:
def max_len_sent():
  max=0
  for i in df1['tweet']:
    if len(i)>max:
      max=len(i)
  return max

max_len_train=max_len_sent()
max_len_train

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

data_to_list = df1['Sentences'].values.tolist()
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data_to_list)
sequences = tokenizer.texts_to_sequences(data_to_list)
train_data = pad_sequences(sequences, maxlen=max_len_train)
print(train_data)
     

In [ ]:
y_train = df1['score'].to_numpy()
y_train=y_train.reshape(len(train_data),1)

In [ ]:
lexicon_dict=pd.read_csv(r"C:\Users\i\Downloads\NRC-Emotion-Lexicon\NRC-Emotion-Lexicon\NRC-Emotion-Lexicon-Wordlevel-v0.92.txt",delimiter = "\t",header=None)


In [ ]:
lexicon_dict = lexicon_dict.rename(columns={'Word': 'word', 'Emotion': 'emotion', 'Value': 'value'})


In [ ]:
lexicon_dict

In [ ]:
lexiconDf = lexicon_dict[['word', 'anger', 'fear', 'joy', 'sadness']]


lexiconDf

In [ ]:
lexiconDf=lexiconDf.reset_index(drop=True)
lexiconDf

In [ ]:
import numpy as np

def generate_lexicon_embeddings(tweet_text, lexicon_df):
    
    
    embeddings = []

    for token in tweet:
        if token in lexiconDf['word'].values:
            token_row = lexiconDf[lexiconDf['word'] == token].iloc[0]
            embeddings.append([token_row['joy'], token_row['sadness']])  # Customize based on the lexicon features you want to use

    embeddings = np.array(embeddings)

   
    aggregated_embedding = np.mean(embeddings, axis=0) 
    return aggregated_embedding


lexicon_train_set = []
for tweet in df1['tweet']:
    embedding = generate_lexicon_embeddings(tweet, lexiconDf)
    lexicon_train_set.append(embedding)

lexicon_train_set = np.array(lexicon_train_set)


In [ ]:
from gensim.models import Word2Vec
sentences = df1['tweet'].tolist()
model = Word2Vec(sentences, size=300, window=5, min_count=1)


model.save("word2vec_model.bin")


In [ ]:
def extract_tweet_features(tweet):
    feature_vector = np.zeros(model.vector_size)  # Initialize an array of zeros with the same dimension as word vectors
    
    for word in tweet:
        if word in model.wv:
            feature_vector += model.wv[word]  # Add the word vector to the feature vector
    
    return feature_vector

# Apply the feature extraction function to each tweet in the DataFrame
df1['tweet_features'] = df1['tweet'].apply(extract_tweet_features)


In [ ]:
lexicon_train_set

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np


X_train = np.column_stack((lexicon_train_set, train_data))


model = LinearRegression()

model.fit(X_train, y_train)


In [ ]:
df1